In [1]:
import sys
sys.path.append('../')
sys.path.append('/Users/sdbykov/work/my_packages/nway')

In [2]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import cat2hpx, pandas_to_fits, my_scaler_forward
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import astropy.io.fits as fits
import healpy as hp
from tensorflow import keras

import nway_python_api as nway
%matplotlib inline

This is NWAY Python API
nwaylib file /Users/sdbykov/work/my_packages/nway/nwaylib/__init__.py


# LH DESI catalog

All DESI objects within LH area + a little boundary


In [3]:
desi_cat_orig = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi_cat_orig = desi_cat_orig.replace([np.inf, -np.inf], np.nan)
print(len(desi_cat_orig), 'original objects')


2418574 original objects


countring missing data

In [4]:
print('original catalog:')

print(len_cat:=len(desi_cat_orig))
print("NOT missing any of ['dered_mag_g', 'dered_mag_r', 'dered_mag_z']:")
print(m1:=len(desi_cat_orig.dropna(subset = ['dered_mag_g', 'dered_mag_r', 'dered_mag_z'], how = 'any')))
print(f"({m1/len_cat*100:.2f}%)")

print("NOT missing any of ['dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2']:")
print(m2:=len(desi_cat_orig.dropna(subset = ['dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2'], how = 'any')))
print(f"({m2/len_cat*100:.2f}%)")

print("NOT missing any of ['dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2', 'dered_mag_w3', 'dered_mag_w4']:")
print(m3:=len(desi_cat_orig.dropna(subset = ['dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2', 'dered_mag_w3', 'dered_mag_w4'], how = 'any')))
print(f"({m3/len_cat*100:.2f}%)")


original catalog:
2418574
NOT missing any of ['dered_mag_g', 'dered_mag_r', 'dered_mag_z']:
2347706
(97.07%)
NOT missing any of ['dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2']:
1684838
(69.66%)
NOT missing any of ['dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2', 'dered_mag_w3', 'dered_mag_w4']:
462226
(19.11%)


filtering the catalog

In [5]:
desi_cat = desi_cat_orig.copy()
desi_cat = desi_cat.sort_values(by=['flux_g'], ascending=[False])
desi_cat = desi_cat.drop_duplicates(subset=['ra', 'dec'], keep='first')

tm_desi_id=desi_cat['release'].astype(str)+'_'+desi_cat['brickid'].astype(str)+'_'+desi_cat['objid'].astype(str)
desi_cat['desi_id'] = tm_desi_id

columns_to_retain = ['desi_id', 'ra', 'dec', 'dered_mag_g', 'dered_mag_r', 'dered_mag_z', 'dered_mag_w1', 'dered_mag_w2', 'dered_mag_w3', 'dered_mag_w4'] 
desi_cat = desi_cat[columns_to_retain]

#TODO mag_g -> dered_mag_g, etc
desi_cat.rename(columns={'dered_mag_g':'mag_g', 'dered_mag_r': 'mag_r', 'dered_mag_z': 'mag_z', 'dered_mag_w1': 'mag_w1', 'dered_mag_w2': 'mag_w2', 'dered_mag_w3': 'mag_w3', 'dered_mag_w4': 'mag_w4'}, inplace=True) 

desi_cat.reset_index(inplace=True, drop = True)
desi_cat.head(5)

,desi_id,ra,dec,mag_g,mag_r,mag_z,mag_w1,mag_w2,mag_w3,mag_w4
0,9011_606802_2605,165.460954,56.382573,2.915254,2.087654,1.722280,5.013193,5.196579,7.574195,8.353606
1,9011_605177_2992,157.655218,55.980398,4.877345,4.564330,4.505185,6.241055,6.745312,8.797101,9.748060
2,9011_608375_2337,158.790917,57.082795,5.150754,5.013778,5.079414,6.963816,7.265361,9.411846,10.460558
3,9011_609165_2169,160.930717,57.198968,5.623667,5.891215,6.298098,8.569067,9.066921,11.004789,12.082440
4,9011_615262_2652,162.848613,59.319899,5.885894,5.040292,4.663559,5.754431,6.882899,8.257881,9.052430


Add color columns to the catalog

In [6]:
desi_cat['col_gr'] = desi_cat['mag_g'] - desi_cat['mag_r']
desi_cat['col_rz'] = desi_cat['mag_r'] - desi_cat['mag_z']
desi_cat['col_gz'] = desi_cat['mag_g'] - desi_cat['mag_z']


desi_cat['col_zw1'] = desi_cat['mag_z'] - desi_cat['mag_w1']
desi_cat['col_rw2'] = desi_cat['mag_r'] - desi_cat['mag_w2']
desi_cat['col_w1w2'] = desi_cat['mag_w1'] - desi_cat['mag_w2']

desi_cat['col_zw3'] = desi_cat['mag_z'] - desi_cat['mag_w3']
desi_cat['col_rw4'] = desi_cat['mag_r'] - desi_cat['mag_w4']
desi_cat['col_w3w4'] = desi_cat['mag_w3'] - desi_cat['mag_w4']

## Dealing with missing data -> assigning prior type

In [7]:
desi_cat['prior_type'] = 'None'

mask = ~(desi_cat.mag_g.isna()) & ~(desi_cat.mag_r.isna()) & ~(desi_cat.mag_z.isna())
desi_cat.loc[mask, 'prior_type'] = 'grz'
mask = mask & ~(desi_cat.mag_w1.isna()) & ~(desi_cat.mag_w2.isna()) 
desi_cat.loc[mask, 'prior_type'] = 'grzw1w2'
mask = mask & ~(desi_cat.mag_w3.isna()) & ~(desi_cat.mag_w4.isna())
desi_cat.loc[mask, 'prior_type'] = 'grzw1w2w3w4'

#desi_cat_scaled_imputed.loc[desi_cat_scaled_imputed.prior_type==prior, features_cols] = -99



In [8]:
print(desi_cat.prior_type.value_counts())
print(desi_cat.prior_type.value_counts(normalize=1))

grzw1w2        1222612
grz             662868
grzw1w2w3w4     462226
None             70868
Name: prior_type, dtype: int64
grzw1w2        0.505509
grz            0.274074
grzw1w2w3w4    0.191115
None           0.029302
Name: prior_type, dtype: float64


## Adding prior columns

In [9]:
desi_cat_scaled = my_scaler_forward(desi_cat) #scale the data for KERAS NN applying

In [10]:
#print(pd.DataFrame(desi_cat_scaled.describe().loc[['mean', 'std',  'max', 'min']]).T)
desi_cat_scaled_imputed= desi_cat_scaled.fillna(value = -99)


In [11]:
for prior in ['grzw1w2w3w4', 'grzw1w2', 'grz']:
    nnmag_name = 'nnmag_'+prior
    desi_cat_scaled_imputed[nnmag_name] = -99
    nnmag_name = 'nnmag_'+prior+'_orig'
    desi_cat_scaled_imputed[nnmag_name] = -99




In [12]:
def make_predictions(df, prior_type):
    features_cols_grz = ['mag_g','mag_r','mag_z', 'col_gr', 'col_rz', 'col_gz']
    features_cols_grzw1w2 = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'col_gr', 'col_rz',  'col_gz','col_zw1', 'col_rw2', 'col_w1w2']
    features_cols_grzw1w2w3w4 = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'mag_w3', 'mag_w4', 'col_gr', 'col_rz', 'col_gz','col_zw1', 'col_rw2', 'col_w1w2', 'col_zw3', 'col_rw4', 'col_w3w4']
    print('Calculating predictions for prior_type: '+prior_type)
    if prior_type == 'grzw1w2':
        features_cols = features_cols_grzw1w2
    elif prior_type == 'grz':
        features_cols = features_cols_grz
    elif prior_type == 'grzw1w2w3w4':
        features_cols = features_cols_grzw1w2w3w4
    else:
        raise ValueError('prior_type type not recognized')

        
    mask = df.prior_type==prior_type
    df_tmp = df[mask][features_cols]


    nnmag_name = 'nnmag_'+prior_type
    model = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_csc-desi-photo-prior/model/'+nnmag_name+'.keras_nn')
    X_predict = df_tmp.values
    y_predict = model.predict(X_predict)

    df.loc[mask, nnmag_name] = y_predict
    df.loc[mask, nnmag_name+'_orig'] = y_predict

    if prior_type!='grz':
        print('--Calculating sub-predictions for prior_type: '+'grz')

        nnmag_name_grz = 'nnmag_'+'grz'
        model_grz = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_csc-desi-photo-prior/model/'+nnmag_name_grz+'.keras_nn')
        X_predict_grz =  df[mask][features_cols_grz].values
        y_predict_grz = model_grz.predict(X_predict_grz)

        df.loc[mask, nnmag_name_grz+'_orig'] = y_predict_grz


    if prior_type=='grzw1w2w3w4':
        print('--Calculating sub-predictions for prior_type: '+'grzw1w2')

        nnmag_name_grzw1w2 = 'nnmag_'+'grzw1w2'
        model_grzw1w2 = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_csc-desi-photo-prior/model/'+nnmag_name_grzw1w2+'.keras_nn')
        X_predict_grzw1w2 =  df[mask][features_cols_grzw1w2].values
        y_predict_grzw1w2 = model_grzw1w2.predict(X_predict_grzw1w2)

        df.loc[mask, nnmag_name_grzw1w2+'_orig'] = y_predict_grzw1w2


    return df

for prior_type in ['grzw1w2w3w4', 'grzw1w2', 'grz']:
    desi_cat_scaled_imputed = make_predictions(desi_cat_scaled_imputed, prior_type)
   

Calculating predictions for prior_type: grzw1w2w3w4
14445/14445 [==============================] - 12s 798us/step
--Calculating sub-predictions for prior_type: grz
14445/14445 [==============================] - 11s 750us/step
--Calculating sub-predictions for prior_type: grzw1w2
14445/14445 [==============================] - 12s 829us/step
Calculating predictions for prior_type: grzw1w2
38207/38207 [==============================] - 35s 906us/step
--Calculating sub-predictions for prior_type: grz
38207/38207 [==============================] - 33s 851us/step
Calculating predictions for prior_type: grz
20715/20715 [==============================] - 17s 838us/step


In [13]:
desi_cat_scaled_imputed_no_nnmag_grz = desi_cat_scaled_imputed.copy() # this is to test the case when i ignore nnmag-grz prior and left it as is, i.e. only grzw1w2(w3w4) trained priors are used


In [14]:
for prior in ['grzw1w2w3w4', 'grzw1w2', 'grz']:
    if prior == 'grzw1w2':
        features_cols = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'col_gr', 'col_rz',  'col_gz','col_zw1', 'col_rw2', 'col_w1w2']
    elif prior == 'grz':
        features_cols = ['mag_g','mag_r','mag_z', 'col_gr', 'col_rz', 'col_gz']
    elif prior == 'grzw1w2w3w4':
        features_cols = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'mag_w3', 'mag_w4', 'col_gr', 'col_rz', 'col_gz','col_zw1', 'col_rw2', 'col_w1w2', 'col_zw3', 'col_rw4', 'col_w3w4']

    mask = desi_cat_scaled_imputed.prior_type==prior
    desi_cat_scaled_imputed.loc[mask, features_cols] = -99


In [15]:


for prior in ['grzw1w2w3w4', 'grzw1w2']:
    if prior == 'grzw1w2':
        features_cols = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'col_gr', 'col_rz',  'col_gz','col_zw1', 'col_rw2', 'col_w1w2']
    elif prior == 'grzw1w2w3w4':
        features_cols = ['mag_g','mag_r','mag_z','mag_w1','mag_w2', 'mag_w3', 'mag_w4', 'col_gr', 'col_rz', 'col_gz','col_zw1', 'col_rw2', 'col_w1w2', 'col_zw3', 'col_rw4', 'col_w3w4']

    mask = desi_cat_scaled_imputed_no_nnmag_grz.prior_type==prior
    desi_cat_scaled_imputed_no_nnmag_grz.loc[mask, features_cols] = -99


In [16]:
photo_columns = ['desi_id', 'mag_g', 'mag_r', 'mag_z', 'mag_w1', 'mag_w2',
       'mag_w3', 'mag_w4', 'col_gr', 'col_rz', 'col_gz', 'col_zw1', 'col_rw2',
       'col_w1w2', 'col_zw3', 'col_rw4', 'col_w3w4',]

desi_cat_scaled_imputed = desi_cat_scaled_imputed.merge(desi_cat[photo_columns], on='desi_id', how='left', suffixes=('', '_orig'))

desi_cat_scaled_imputed_no_nnmag_grz = desi_cat_scaled_imputed_no_nnmag_grz.merge(desi_cat[photo_columns], on='desi_id', how='left', suffixes=('', '_orig'))
desi_cat_scaled_imputed.head(10)

,desi_id,ra,dec,mag_g,mag_r,mag_z,mag_w1,mag_w2,mag_w3,mag_w4,col_gr,col_rz,col_gz,col_zw1,col_rw2,col_w1w2,col_zw3,col_rw4,col_w3w4,prior_type,nnmag_grzw1w2w3w4,nnmag_grzw1w2w3w4_orig,nnmag_grzw1w2,nnmag_grzw1w2_orig,nnmag_grz,nnmag_grz_orig,mag_g_orig,mag_r_orig,mag_z_orig,mag_w1_orig,mag_w2_orig,mag_w3_orig,mag_w4_orig,col_gr_orig,col_rz_orig,col_gz_orig,col_zw1_orig,col_rw2_orig,col_w1w2_orig,col_zw3_orig,col_rw4_orig,col_w3w4_orig
0,9011_606802_2605,165.460954,56.382573,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.973107,0.973107,-99.0,0.978131,-99.0,0.842576,2.915254,2.087654,1.722280,5.013193,5.196579,7.574195,8.353606,0.827600,0.365374,1.192974,-3.290913,-3.108925,-0.183386,-5.851915,-6.265952,-0.779411
1,9011_605177_2992,157.655218,55.980398,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.973069,0.973069,-99.0,0.978469,-99.0,0.842640,4.877345,4.564330,4.505185,6.241055,6.745312,8.797101,9.748060,0.313015,0.059145,0.372160,-1.735870,-2.180982,-0.504257,-4.291916,-5.183730,-0.950959
2,9011_608375_2337,158.790917,57.082795,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.973065,0.973065,-99.0,0.978488,-99.0,0.842942,5.150754,5.013778,5.079414,6.963816,7.265361,9.411846,10.460558,0.136976,-0.065636,0.071340,-1.884402,-2.251583,-0.301545,-4.332432,-5.446780,-1.048712
3,9011_609165_2169,160.930717,57.198968,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.972578,0.972578,-99.0,0.976215,-99.0,0.853456,5.623667,5.891215,6.298098,8.569067,9.066921,11.004789,12.082440,-0.267548,-0.406883,-0.674431,-2.270969,-3.175706,-0.497854,-4.706691,-6.191225,-1.077651
4,9011_615262_2652,162.848613,59.319899,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.972705,0.972705,-99.0,0.977987,-99.0,0.842574,5.885894,5.040292,4.663559,5.754431,6.882899,8.257881,9.052430,0.845602,0.376733,1.222335,-1.090872,-1.842607,-1.128468,-3.594322,-4.012138,-0.794549
5,9011_606796_2976,162.795345,56.582266,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.972943,0.972943,-99.0,0.978410,-99.0,0.842573,6.005697,5.146538,4.761223,5.769194,6.643266,8.270793,9.135516,0.859159,0.385315,1.244474,-1.007971,-1.496728,-0.874072,-3.509570,-3.988978,-0.864723
6,9011_603543_1943,155.273704,55.567940,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.973275,0.973275,-99.0,0.978797,-99.0,0.952518,6.340635,10.343140,12.109075,10.872367,11.500216,13.251790,14.362062,-4.002505,-1.765935,-5.768440,1.236708,-1.157076,-0.627849,-1.142715,-4.018922,-1.110272
7,9011_618180_2571,162.799635,60.168878,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.973285,0.973285,-99.0,0.978797,-99.0,0.951018,6.535789,9.906864,13.324006,10.599302,10.937230,12.706153,13.247221,-3.371075,-3.417142,-6.788217,2.724704,-1.030366,-0.337928,0.617853,-3.340357,-0.541068
8,9011_617452_3218,159.582572,60.123663,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.972510,0.972510,-99.0,0.977902,-99.0,0.842844,6.850522,6.599732,6.584245,8.220688,8.769516,11.059014,12.310115,0.250790,0.015487,0.266277,-1.636443,-2.169784,-0.548828,-4.474769,-5.710383,-1.251101
9,9011_606789_639,159.335279,56.431316,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,grzw1w2w3w4,0.973076,0.973076,-99.0,0.978626,-99.0,0.842571,6.934036,6.065136,5.672889,6.589483,6.974699,9.045024,10.134847,0.868900,0.392247,1.261147,-0.916594,-0.909563,-0.385216,-3.372135,-4.069711,-1.089823


In [17]:
desi_cat_scaled_imputed.groupby('prior_type')[['nnmag_grzw1w2w3w4', 'nnmag_grzw1w2', 'nnmag_grz']].agg(['max'])


,nnmag_grzw1w2w3w4,nnmag_grzw1w2,nnmag_grz
,max,max,max
prior_type,,,
None,-99.000000,-99.000000,-99.000000
grz,-99.000000,-99.000000,0.952644
grzw1w2,-99.000000,0.978783,-99.000000
grzw1w2w3w4,0.973285,-99.000000,-99.000000


In [18]:
desi_cat_scaled_imputed.groupby('prior_type')[['nnmag_grzw1w2w3w4_orig', 'nnmag_grzw1w2_orig', 'nnmag_grz_orig']].agg(['max'])


,nnmag_grzw1w2w3w4_orig,nnmag_grzw1w2_orig,nnmag_grz_orig
,max,max,max
prior_type,,,
None,-99.000000,-99.000000,-99.000000
grz,-99.000000,-99.000000,0.952644
grzw1w2,-99.000000,0.978783,0.952686
grzw1w2w3w4,0.973285,0.978797,0.952691


## area and plotting

In [19]:
#desi_hp_map = cat2hpx(desi_cat_scaled_imputed['ra'], desi_cat_scaled_imputed['dec'], nside=1024, radec=True)

In [22]:
%matplotlib inline

try:
    hp.gnomview(desi_hp_map, rot = [150.23,52.19], xsize = 400)
    DESI_SKYAREA = 4*np.pi*(np.rad2deg(1))**2 * np.sum(desi_hp_map>=1)/len(desi_hp_map)
except:
    DESI_SKYAREA = 41.728630922842704
print(DESI_SKYAREA, 'sq. deg.')

desi_sky_dens = len(desi_cat_scaled_imputed)/DESI_SKYAREA
print('DESI sky density:', desi_sky_dens, 'per sq deg')
print('DESI sky density:', desi_sky_dens/(3600), 'per sq. arcmin')
print('DESI sky density:', desi_sky_dens/(3600**2), 'per sq. arcsec')
print(' expected number of DESI sources within 1 arcsec radius', np.pi*1**2 * desi_sky_dens/(3600**2))
print(' expected number of DESI sources within 5 arcsec radius', np.pi*5**2 * desi_sky_dens/(3600**2))
print(' expected number of DESI sources within 10 arcsec radius', np.pi*10**2 * desi_sky_dens/(3600**2))



41.728630922842704 sq. deg.
DESI sky density: 57959.58186291816 per sq deg
DESI sky density: 16.0998838508106 per sq. arcmin
DESI sky density: 0.0044721899585585 per sq. arcsec
 expected number of DESI sources within 1 arcsec radius 0.014049799119265424
 expected number of DESI sources within 5 arcsec radius 0.3512449779816356
 expected number of DESI sources within 10 arcsec radius 1.4049799119265425


In [23]:
pandas_to_fits(dataframe = desi_cat_scaled_imputed, 
                filename = 'desi_lh.fits',
                table_header_name = 'DESI',
                sky_area_deg2 = DESI_SKYAREA)

pandas_to_fits(dataframe = desi_cat_scaled_imputed_no_nnmag_grz,
                filename = 'desi_lh_no_nnmag_grz.fits',
                table_header_name = 'DESI',
                sky_area_deg2 = DESI_SKYAREA)

In [24]:
desi_cat_scaled_imputed.query("prior_type == 'grzw1w2w3w4'")[['nnmag_grz_orig','nnmag_grzw1w2_orig', 'nnmag_grzw1w2w3w4_orig']].corr()

,nnmag_grz_orig,nnmag_grzw1w2_orig,nnmag_grzw1w2w3w4_orig
nnmag_grz_orig,1.000000,0.429686,0.325436
nnmag_grzw1w2_orig,0.429686,1.000000,0.941339
nnmag_grzw1w2w3w4_orig,0.325436,0.941339,1.000000


In [25]:
desi_cat_scaled_imputed.query("prior_type == 'grzw1w2w3w4'").iloc[0]

desi_id                   9011_606802_2605
ra                              165.460954
dec                              56.382573
mag_g                                -99.0
mag_r                                -99.0
mag_z                                -99.0
mag_w1                               -99.0
mag_w2                               -99.0
mag_w3                               -99.0
mag_w4                               -99.0
col_gr                               -99.0
col_rz                               -99.0
col_gz                               -99.0
col_zw1                              -99.0
col_rw2                              -99.0
col_w1w2                             -99.0
col_zw3                              -99.0
col_rw4                              -99.0
col_w3w4                             -99.0
prior_type                     grzw1w2w3w4
nnmag_grzw1w2w3w4                 0.973107
nnmag_grzw1w2w3w4_orig            0.973107
nnmag_grzw1w2                        -99.0
nnmag_grzw1

In [26]:
desi_cat_scaled_imputed_no_nnmag_grz.query("prior_type == 'grzw1w2'").iloc[0]

desi_id                   9011_603560_3953
ra                              162.773019
dec                              55.592529
mag_g                                -99.0
mag_r                                -99.0
mag_z                                -99.0
mag_w1                               -99.0
mag_w2                               -99.0
mag_w3                            0.403153
mag_w4                               -99.0
col_gr                               -99.0
col_rz                               -99.0
col_gz                               -99.0
col_zw1                              -99.0
col_rw2                              -99.0
col_w1w2                             -99.0
col_zw3                          -0.025437
col_rw4                              -99.0
col_w3w4                             -99.0
prior_type                         grzw1w2
nnmag_grzw1w2w3w4                    -99.0
nnmag_grzw1w2w3w4_orig               -99.0
nnmag_grzw1w2                     0.978783
nnmag_grzw1